## Read the pdf file using PyPDFLoader and apply Underwoods list of ocr corrections

In [8]:
# Save start time

print ('Import libraries')
# import libraries
from langchain_community.document_loaders import UnstructuredEPubLoader
from langchain.document_loaders import PyPDFLoader
import pandas as pd
import re
import os
import csv
import json
from tqdm import tqdm  # Import tqdm for progress tracking
import time

print ('Start time')
start_time = time.time()

####### PDF loader #############
# load pdf
# Move to pdf folder
os.chdir(r'.\literature pdf')

# load text from pdf
pdf_file = '1903 Thirty seasons in Scandinavia.pdf'
book_title = pdf_file[:-4]
print (f'Start processing "{book_title}".\nWait a few seconds.')
loader = PyPDFLoader(pdf_file)
# Load PDF pages
pdf_pages = list(tqdm(loader.load(), desc="Loading PDF pages", colour='yellow'))

# Add one to get a reference back to the right page in pdf file
def get_pdf_page_no(pageNo):
    return pageNo + 1

# Extract the name of the source
def get_page_source(pageSource):
    return pageSource.replace('.pdf', '').split('\\')[-1]

print ('Preprocess text')
# preprocess text
def preproces_text(text_string):
    # Store text data in variable
    # Remove newlines, replace apostrophe s with s, remove quotation marks
    text_string_replace = text_string.replace(r' \xad\n', '').replace(r'\xad\n', '') \
        .replace(r'\xad', '').replace(r'\xad ', '') \
        .replace(r'-\n', '').replace('—', ' ').replace(r'\n', ' ') \
        .replace("'s", "s").replace('"', ' ').replace("'", " ") \
        .replace('.', ' . ').replace(' .', ' . ').replace(',', ' ,') \
        .replace('!', ' !').replace('?', ' ?')

    # scrub for multiple signs
    # but keep full stops, commas, exclamation, and question marks
    text_string_re = re.sub(r':|;|\(|\)|\||\+|\"|‘|’|“|”|\’|…|\-|–|—|\$|&|\*|>|<|\/|\[|\]|»|«', '', text_string_replace)

    # Remove numbers and words containing numbers
    text_string_no_numbers = re.sub(r'\b\w*\d\w*\b', '', text_string_re)
    # Remove multiple whitespaces
    text_string_sub_additional_white_space = re.sub(r'\s+', ' ', text_string_no_numbers)

    return text_string_sub_additional_white_space

# Extract page info
def extract_page_data(pdf_pages):
    # prepare empty lists
    content_list = []
    semi_clean_text_list = []
    page_no_list = []
    source_list = []

    for i in tqdm(pdf_pages, desc="Extracting and cleaning data", colour='yellow'):
        # take page number
        page = i

        # return the text content, preproces text, and metadata (source and page no.)
        pageContent = page.page_content
        semi_clean_text = preproces_text(pageContent)
        pageNo = page.metadata['page']
        pageSource = page.metadata['source']

        # append to lists
        content_list.append(pageContent)
        semi_clean_text_list.append(semi_clean_text)
        page_no_list.append(get_pdf_page_no(pageNo))
        source_list.append(get_page_source(pageSource))

    # output data
    return content_list, semi_clean_text_list, page_no_list, source_list

# return lists of content, page numbers, and source
C, SC, P, S = extract_page_data(pdf_pages)

# send lists to a dataframe
df = pd.DataFrame({'content': C, 'source': S, 'element_no': P, 'preprocessed_content': SC})


#######################

# go back to main folder
os.chdir('..')


print('Starting ocr correction - loading correction rules')
# Read the correction rules from the file
with open('Underwoods_CorrectionRules.txt', 'r', encoding='utf-8') as f:
    CorrectionRules_string = f.read()

# Process the correction rules
CorrectionRules_list = CorrectionRules_string.split('\n')
list_of_CorrectionRules_list = [i.split('\t') for i in CorrectionRules_list]
better_list_of_CorrectionRules = [i[0:2] for i in list_of_CorrectionRules_list]

# Add additional pairs
observations_to_add = [["Bruffels", 'Brussels'], ["fix", 'six'], ['Elsinore', 'Helsingør']]
better_list_of_CorrectionRules.extend(observations_to_add)

# Build correction dictionary
correction_dict = dict(better_list_of_CorrectionRules)



print('Ocr correction')

def ocr_post_correction(text):
    # Prepare for correction
    ### Keep track of the conditions of the words (capital, title, upper,lower)
    tokens = text.split()

    # Track the original case of each word
    original_case = [i for i in tokens]
    clean_text_list = [i.lower() for i in tokens]

    # Initialize the list to store corrected words and a dictionary to track corrections
    new_word_list = []
    corrections_tracker = {}

    # Iterate through the clean_text_list word list
    for index, word in enumerate(clean_text_list):
        # Check if the word has a correction in the dictionary
        if word in correction_dict:
            # If it does, append the corrected word to the new list and track the correction
            corrected_word = correction_dict[word]
            # Restore the original case of the word
            if original_case[index].istitle():
                corrected_word = corrected_word.capitalize()
            elif original_case[index].isupper():
                corrected_word = corrected_word.upper()
            new_word_list.append(corrected_word)
            corrections_tracker[word] = corrected_word

            # Extract context around the corrected word
            start_index = max(0, index - 10)
            end_index = min(len(clean_text_list), index + 11)
            context = clean_text_list[start_index:end_index]

            # Store the context with the correction
            corrections_tracker[word] = {
                'corrected_word': corrected_word,
                'context': ' '.join(context)
            }
        else:
            # If it doesn't, append the original word
            new_word_list.append(original_case[index])

    # Send data back to text
    new_corrected_text = ' '.join(new_word_list)

    return corrections_tracker, new_corrected_text

# Use the function and make two new columns to add to the dataframe
df['corrections_tracker'], df['ocr_corrected_text'] = zip(*tqdm(df['preprocessed_content'].apply(ocr_post_correction), desc="Applying OCR post-correction", colour='yellow'))

print ('Saving data')
# Save the df as a csv file
os.chdir(r'.\literature csv')
df.to_csv(f'{book_title}.csv', index=False)


# Save txt versions
os.chdir('..')
os.chdir(r'.\literature txt')

# Save text version without spell correction
text = ' '.join(df['preprocessed_content'].to_list())
with open(f'{book_title} without ocr correction.txt', 'w', encoding='utf-8') as f:
    f.write(text)

# Save text version with spell correction
# Join the corrected words back into a string
ocr_corrected_text = ' '.join(df['ocr_corrected_text'].to_list())
with open(f'{book_title} with ocr correction.txt', 'w', encoding='utf-8') as f:
    f.write(ocr_corrected_text)


########
# Save ocr corrections
print ('Save ocr corrections')
os.chdir('..')
os.chdir(r'.\ocr_correction_tracker')
obs_corrections_tracker = []
for i,j in enumerate(df['corrections_tracker'].to_list()):
    if j:
        observation = f"Index no.: {str(i)}. OCR observation: {str(j)}"
        observation = observation.replace('{', '').replace('}', '')
        obs_corrections_tracker.append(observation)

ocr_observations = '\n\n'.join(obs_corrections_tracker)

with open(f'{book_title} ocr corrections.txt', 'w', encoding='utf-8') as f:
    f.write(ocr_observations)

##############



# change to main folder
os.chdir('..')

print ('Script done.')
# Record the end time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the elapsed time
print(f"Time taken to run the script: {elapsed_time:.6f} seconds")

Import libraries
Start time
Start processing "1903 Thirty seasons in Scandinavia".
Wait a few seconds.


Loading PDF pages: 100%|██████████| 337/337 [00:00<?, ?it/s]


Preprocess text


Extracting and cleaning data: 100%|██████████| 337/337 [00:00<00:00, 8227.62it/s]


Starting ocr correction - loading correction rules
Ocr correction


Applying OCR post-correction: 100%|██████████| 337/337 [00:00<?, ?it/s]

Saving data
Save ocr corrections
Script done.
Time taken to run the script: 6.405390 seconds


In [9]:
df

,content,source,element_no,preprocessed_content,corrections_tracker,ocr_corrected_text
0,Digitaliseret af | Digitised by\nForfatter(e) ...,1903 Thirty seasons in Scandinavia,1,Digitaliseret af Digitised by Forfatter e Auth...,{},Digitaliseret af Digitised by Forfatter e Auth...
1,"ThirtÖmws\n■\n,N\n^(ANDINAVH/ w\nytJ,■",1903 Thirty seasons in Scandinavia,2,"ThirtÖmws ■ ,N ^ ANDINAVH w ytJ ,■",{},"ThirtÖmws ■ ,N ^ ANDINAVH w ytJ ,■"
2,Det Kgl. Bibliotek\n130025471602,1903 Thirty seasons in Scandinavia,3,Det Kgl . Bibliotek 130025471602,{},Det Kgl . Bibliotek 130025471602
3,,1903 Thirty seasons in Scandinavia,4,,{},
4,,1903 Thirty seasons in Scandinavia,5,,{},
...,...,...,...,...,...,...
332,,1903 Thirty seasons in Scandinavia,333,,{},
333,,1903 Thirty seasons in Scandinavia,334,,{},
334,,1903 Thirty seasons in Scandinavia,335,,{},
335,,1903 Thirty seasons in Scandinavia,336,,{},


In [4]:
print ('Import libraries')
# import libraries
from langchain_community.document_loaders import UnstructuredEPubLoader
from langchain.document_loaders import PyPDFLoader
import pandas as pd
import re
import os
import csv
import json
from tqdm import tqdm  # Import tqdm for progress tracking
import time

Import libraries


In [5]:
path_to_folder = r'C:\Users\lakj\Documents\GitHub\nordic travel literature\literature pdf'
no_of_files = len(os.listdir(path_to_folder))
files = os.listdir(path_to_folder)

In [6]:

def input_is_folder(pdf_file):

    
    print ('Start time')
    start_time = time.time()
    
    ####### PDF loader #############
    # load pdf
    # Move to pdf folder
    os.chdir(r'.\literature pdf')
    
    # load text from pdf
    #pdf_file = 'Scenes of travel in Norway J C Phythian.pdf'
    book_title = pdf_file[:-4]
    print (f'Start processing "{book_title}".\nWait a few seconds.')
    loader = PyPDFLoader(pdf_file)
    # Load PDF pages
    pdf_pages = list(tqdm(loader.load(), desc="Loading PDF pages", colour='yellow'))
    
    # Add one to get a reference back to the right page in pdf file
    def get_pdf_page_no(pageNo):
        return pageNo + 1
    
    # Extract the name of the source
    def get_page_source(pageSource):
        return pageSource.replace('.pdf', '').split('\\')[-1]
    
    print ('Preprocess text')
    # preprocess text
    def preproces_text(text_string):
        # Store text data in variable
        # Remove newlines, replace apostrophe s with s, remove quotation marks
        text_string_replace = text_string.replace(r' \xad\n', '').replace(r'\xad\n', '') \
            .replace(r'\xad', '').replace(r'\xad ', '') \
            .replace(r'-\n', '').replace('—', ' ').replace(r'\n', ' ') \
            .replace("'s", "s").replace('"', ' ').replace("'", " ") \
            .replace('.', ' . ').replace(' .', ' . ').replace(',', ' ,') \
            .replace('!', ' !').replace('?', ' ?')
    
        # scrub for multiple signs
        # but keep full stops, commas, exclamation, and question marks
        text_string_re = re.sub(r':|;|\(|\)|\||\+|\"|‘|’|“|”|\’|…|\-|–|—|\$|&|\*|>|<|\/|\[|\]|»|«', '', text_string_replace)
    
        # Remove numbers and words containing numbers
        text_string_no_numbers = re.sub(r'\b\w*\d\w*\b', '', text_string_re)
        # Remove multiple whitespaces
        text_string_sub_additional_white_space = re.sub(r'\s+', ' ', text_string_no_numbers)
    
        return text_string_sub_additional_white_space
    
    # Extract page info
    def extract_page_data(pdf_pages):
        # prepare empty lists
        content_list = []
        semi_clean_text_list = []
        page_no_list = []
        source_list = []
    
        for i in tqdm(pdf_pages, desc="Extracting and cleaning data", colour='yellow'):
            # take page number
            page = i
    
            # return the text content, preproces text, and metadata (source and page no.)
            pageContent = page.page_content
            semi_clean_text = preproces_text(pageContent)
            pageNo = page.metadata['page']
            pageSource = page.metadata['source']
    
            # append to lists
            content_list.append(pageContent)
            semi_clean_text_list.append(semi_clean_text)
            page_no_list.append(get_pdf_page_no(pageNo))
            source_list.append(get_page_source(pageSource))
    
        # output data
        return content_list, semi_clean_text_list, page_no_list, source_list
    
    # return lists of content, page numbers, and source
    C, SC, P, S = extract_page_data(pdf_pages)
    
    # send lists to a dataframe
    df = pd.DataFrame({'content': C, 'source': S, 'element_no': P, 'preprocessed_content': SC})
    
    
    #######################
    
    # go back to main folder
    os.chdir('..')
    
    
    print('Starting ocr correction - loading correction rules')
    # Read the correction rules from the file
    with open('Underwoods_CorrectionRules.txt', 'r', encoding='utf-8') as f:
        CorrectionRules_string = f.read()
    
    # Process the correction rules
    CorrectionRules_list = CorrectionRules_string.split('\n')
    list_of_CorrectionRules_list = [i.split('\t') for i in CorrectionRules_list]
    better_list_of_CorrectionRules = [i[0:2] for i in list_of_CorrectionRules_list]
    
    # Add additional pairs
    observations_to_add = [["Bruffels", 'Brussels'], ["fix", 'six'], ['Elsinore', 'Helsingør']]
    better_list_of_CorrectionRules.extend(observations_to_add)
    
    # Build correction dictionary
    correction_dict = dict(better_list_of_CorrectionRules)
    
    
    
    print('Ocr correction')
    
    def ocr_post_correction(text):
        # Prepare for correction
        ### Keep track of the conditions of the words (capital, title, upper,lower)
        tokens = text.split()
    
        # Track the original case of each word
        original_case = [i for i in tokens]
        clean_text_list = [i.lower() for i in tokens]
    
        # Initialize the list to store corrected words and a dictionary to track corrections
        new_word_list = []
        corrections_tracker = {}
    
        # Iterate through the clean_text_list word list
        for index, word in enumerate(clean_text_list):
            # Check if the word has a correction in the dictionary
            if word in correction_dict:
                # If it does, append the corrected word to the new list and track the correction
                corrected_word = correction_dict[word]
                # Restore the original case of the word
                if original_case[index].istitle():
                    corrected_word = corrected_word.capitalize()
                elif original_case[index].isupper():
                    corrected_word = corrected_word.upper()
                new_word_list.append(corrected_word)
                corrections_tracker[word] = corrected_word
    
                # Extract context around the corrected word
                start_index = max(0, index - 10)
                end_index = min(len(clean_text_list), index + 11)
                context = clean_text_list[start_index:end_index]
    
                # Store the context with the correction
                corrections_tracker[word] = {
                    'corrected_word': corrected_word,
                    'context': ' '.join(context)
                }
            else:
                # If it doesn't, append the original word
                new_word_list.append(original_case[index])
    
        # Send data back to text
        new_corrected_text = ' '.join(new_word_list)
    
        return corrections_tracker, new_corrected_text
    
    # Use the function and make two new columns to add to the dataframe
    df['corrections_tracker'], df['ocr_corrected_text'] = zip(*tqdm(df['preprocessed_content'].apply(ocr_post_correction), desc="Applying OCR post-correction", colour='yellow'))
    
    print ('Saving data')
    # Save the df as a csv file
    os.chdir(r'.\literature csv')
    df.to_csv(f'{book_title}.csv', index=False)
    
    
    # Save txt versions
    os.chdir('..')
    os.chdir(r'.\literature txt')
    
    # Save text version without spell correction
    text = ' '.join(df['preprocessed_content'].to_list())
    with open(f'{book_title} without ocr correction.txt', 'w', encoding='utf-8') as f:
        f.write(text)
    
    # Save text version with spell correction
    # Join the corrected words back into a string
    ocr_corrected_text = ' '.join(df['ocr_corrected_text'].to_list())
    with open(f'{book_title} with ocr correction.txt', 'w', encoding='utf-8') as f:
        f.write(ocr_corrected_text)
    
    
    ########
    # Save ocr corrections
    print ('Save ocr corrections')
    os.chdir('..')
    os.chdir(r'.\ocr_correction_tracker')
    obs_corrections_tracker = []
    for i,j in enumerate(df['corrections_tracker'].to_list()):
        if j:
            observation = f"Index no.: {str(i)}. OCR observation: {str(j)}"
            observation = observation.replace('{', '').replace('}', '')
            obs_corrections_tracker.append(observation)
    
    ocr_observations = '\n\n'.join(obs_corrections_tracker)
    
    with open(f'{book_title} ocr corrections.txt', 'w', encoding='utf-8') as f:
        f.write(ocr_observations)
    
    ##############
    
    
    
    # change to main folder
    os.chdir('..')
    
    print ('Script done.')
    # Record the end time
    end_time = time.time()
    
    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    
    # Print the elapsed time
    print(f"Time taken to run the script: {elapsed_time:.6f} seconds")

In [7]:
[input_is_folder(pdf_file) for pdf_file in files]

Start time
Start processing "1772 Travels through Holland Flanders Germany Denmark Sweden Lapland Russia the Ukraine and Poland in the years 1768 1769 and 1770 Vol 1".
Wait a few seconds.


Loading PDF pages: 100%|██████████| 202/202 [00:00<?, ?it/s]


Preprocess text


Extracting and cleaning data: 100%|██████████| 202/202 [00:00<00:00, 4191.32it/s]


Starting ocr correction - loading correction rules
Ocr correction


...

Extracting and cleaning data: 100%|██████████| 401/401 [00:00<00:00, 6599.32it/s]


Starting ocr correction - loading correction rules
Ocr correction


Applying OCR post-correction: 100%|██████████| 401/401 [00:00<?, ?it/s]

Saving data
Save ocr corrections
Script done.
Time taken to run the script: 8.652174 seconds


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]